### Import Libraries 

In [28]:


import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline 


#Need to clean further 
from Standardization import metric_normalizer
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler,OneHotEncoder, OrdinalEncoder






from sklearn.metrics import confusion_matrix,accuracy_score, classification_report  # Import accuracy_score


from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA 
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectPercentile, mutual_info_classif
from sklearn.impute import SimpleImputer

from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict

from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, f1_score, precision_score, recall_score
#formats page
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
display(HTML("<style>.container { width:100% !important; }</style>"))


### Import Dataset

In [3]:
df = pd.read_csv('/home/clivence/base_jupyter/Datadump/Model_Data/GU_Model_Data_V1_16k')

### Data Cleaning

In [4]:
#Filter out negative values - evetually need to research why this is happening 
df  = df[(df['2D Low in Pips'] > 0) | (df['2D Low in Pips'] > 0)] 

df = df[df['Action'] == 'Ultimate Action']
df = df[df['Ticker'] == 'GBP/USD']

df.fillna(0, inplace=True)

#Standardize the dataset
df =  metric_normalizer(df)

#Set date to datetime 
df['Date'] = pd.to_datetime(df['Date'])

#Create Date Derived Features 
df['Trade_Week_Year'] = df['Date'].dt.isocalendar().week
df['Trade_Week_Month'] = (df['Date'].dt.day -1)//7+1
df['Trade_Day_Week'] = df['Date'].dt.weekday + 1

#Create a new feature to identify the status of the previous trade 
df['Previous_Trade_Status'] = df['2D Trade Status'].shift(fill_value=0)


### Data Preprocessing 

In [6]:
#Subset Selection
df = df[['open', 'high', 'low', 'close', 'volume','Trade_Week_Year','Trade_Week_Month','Trade_Day_Week',
       'Day','Month','25EMA', '50EMA', '75EMA','100EMA', '125EMA',
        'Trend Status', 'Spread','5075 Trend Status', '75100 Trend Status',
       '100125 Trend Status', 'Order Type', 
        '%K', '%D', 'k_group', 'ADX', 'ADXR', 'slowk','slowd','CDL2CROWS', 'CDL3BLACKCROWS',
        'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE',
        'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK',
        'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL',
        'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI',
        'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE',
        'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS',
        'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS',
        'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH', 'CDLLADDERBOTTOM',
        'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU', 'CDLMATCHINGLOW', 'CDLMATHOLD',
        'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR', 'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN',
        'CDLRISEFALL3METHODS', 'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSHORTLINE',
        'CDLSPINNINGTOP', 'CDLSTALLEDPATTERN', 'CDLSTICKSANDWICH', 'CDLTAKURI', 'CDLTASUKIGAP',
        'CDLTHRUSTING', 'CDLTRISTAR', 'CDLUNIQUE3RIVER', 'CDLUPSIDEGAP2CROWS', 'CDLXSIDEGAP3METHODS',
        'candle_bullish_score','candle_bearish_score','Previous_Trade_Status','2D Trade Status']]

In [23]:
# set training, test, and validation set size 
total_len = len(df)
training_size = int(total_len * .8)
test_size = int(total_len * .1)
validation_size = total_len - training_size - test_size

# Split indices
training_idx = df.index[:training_size]
test_idx = df.index[training_size:training_size + test_size]
validation_idx = df.index[-validation_size:]

# Verify lengths
assert len(training_idx) == training_size
assert len(test_idx) == test_size
assert len(validation_idx) == validation_size

# Create training, test, and validation df 
training_df = df.loc[training_idx]
test_df = df.loc[test_idx]
validation_df = df.loc[validation_idx]

# Print lengths of the three DataFrames
print(f"Training: {len(training_df)}")
print(f"Test: {len(test_df)}")
print(f"Validation: {len(validation_df)}")

Training: 386
Test: 48
Validation: 49


In [24]:
X_train = training_df[['open', 'high', 'low', 'close', 'volume','Trade_Week_Year','Trade_Week_Month','Trade_Day_Week',
       'Day','Month','25EMA', '50EMA', '75EMA','100EMA', '125EMA',
        'Trend Status', 'Spread','5075 Trend Status', '75100 Trend Status',
       '100125 Trend Status', 'Order Type', 
        '%K', '%D', 'k_group', 'ADX', 'ADXR', 'slowk','slowd','CDL2CROWS', 'CDL3BLACKCROWS',
        'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE',
        'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK',
        'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL',
        'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI',
        'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE',
        'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS',
        'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS',
        'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH', 'CDLLADDERBOTTOM',
        'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU', 'CDLMATCHINGLOW', 'CDLMATHOLD',
        'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR', 'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN',
        'CDLRISEFALL3METHODS', 'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSHORTLINE',
        'CDLSPINNINGTOP', 'CDLSTALLEDPATTERN', 'CDLSTICKSANDWICH', 'CDLTAKURI', 'CDLTASUKIGAP',
        'CDLTHRUSTING', 'CDLTRISTAR', 'CDLUNIQUE3RIVER', 'CDLUPSIDEGAP2CROWS', 'CDLXSIDEGAP3METHODS',
        'candle_bullish_score','candle_bearish_score','Previous_Trade_Status','2D Trade Status']]
y_train = training_df['2D Trade Status']

X_test = test_df[['open', 'high', 'low', 'close', 'volume','Trade_Week_Year','Trade_Week_Month','Trade_Day_Week',
       'Day','Month','25EMA', '50EMA', '75EMA','100EMA', '125EMA',
        'Trend Status', 'Spread','5075 Trend Status', '75100 Trend Status',
       '100125 Trend Status', 'Order Type', 
        '%K', '%D', 'k_group', 'ADX', 'ADXR', 'slowk','slowd','CDL2CROWS', 'CDL3BLACKCROWS',
        'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE',
        'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK',
        'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL',
        'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI',
        'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE',
        'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS',
        'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS',
        'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH', 'CDLLADDERBOTTOM',
        'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU', 'CDLMATCHINGLOW', 'CDLMATHOLD',
        'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR', 'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN',
        'CDLRISEFALL3METHODS', 'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSHORTLINE',
        'CDLSPINNINGTOP', 'CDLSTALLEDPATTERN', 'CDLSTICKSANDWICH', 'CDLTAKURI', 'CDLTASUKIGAP',
        'CDLTHRUSTING', 'CDLTRISTAR', 'CDLUNIQUE3RIVER', 'CDLUPSIDEGAP2CROWS', 'CDLXSIDEGAP3METHODS',
        'candle_bullish_score','candle_bearish_score','Previous_Trade_Status','2D Trade Status']]
y_test = test_df['2D Trade Status']

In [26]:
#Create a list of columns to encode 
cat_cols_to_encode = ['Order Type']
#Create a list of cols for ordinal encoding 
cat_cols_for_ordinal_encoding = ['Trend Status','5075 Trend Status', '75100 Trend Status',
                    '100125 Trend Status','k_group']

#Create a list of cols to scale 
num_cols_to_scale = ['volume']

# Create Column Transformer with appropriate preprocessing for numerical and categorical data
Preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
            ('scaler', MinMaxScaler())]), num_cols_to_scale),
        ('cat_onehot', OneHotEncoder(drop='first', sparse_output=False), cat_cols_to_encode),
        ('cat_ordinal', OrdinalEncoder(), cat_cols_for_ordinal_encoding)
    ],
    remainder='passthrough')

# Preprocessor.set_output(transform='pandas')

In [30]:
def enhanced_metrics( predictions,test=y_test):
    '''This function calculates and returns various evaluation metrics based on predicted values and true labels.
    Returns:
- output: str
  A string containing the calculated evaluation metrics.

Metrics included:
- Accuracy: The proportion of correctly classified samples.
- F1 Score: The harmonic mean of precision and recall, indicating overall model performance.
- Recall: The proportion of true positive samples correctly classified.
- Precision: The proportion of predicted positive samples that are actually positive.
    
'''
    Accuracy = accuracy_score(test, predictions)
    f_score = f1_score(test, predictions)
    recall_ = recall_score(test, predictions)
    precision = precision_score(test, predictions)
    Out_put = f"Accuracy: {(Accuracy.round(decimals=3))*100}\n" + \
            f"F1 Score: {(f_score.round(decimals=3))*100}\n" + \
            f"Recall: {(recall_.round(decimals=3))*100}\n" + \
            f"Precision: {(precision.round(decimals=3))*100}" 
    return Out_put

### Base Models with No Parameter

In [35]:
#Create PiPeline to be used in Model
Base_LR_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('RF_Model',LogisticRegression())])

# Perform cross-validation
Base_LR_Model_Pipeline.fit(X_train, y_train)
Base_LR_Model_Predictions = Base_LR_Model_Pipeline.predict(X_test)

# Display the metrics
accuracy_score(test, Base_LR_Model_Predictions)

/home/clivence/base_jupyter/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


InvalidParameterError: The 'y_true' parameter of accuracy_score must be an array-like or a sparse matrix. Got 48 instead.

In [9]:
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostClassifier



# #Create Pipeline to be used in SVC Model
# Base_SVC_Model_Pipeline = Pipeline(steps=[
#     ('Preprocessor', Preprocessor),
#     ('Feature_Selection', Selection),
#     ('SVC_Model',SVC())])

# # Perform cross-validation
# SVC_CV_accuracy = cross_val_score(Base_SVC_Model_Pipeline, X, Y, cv=5, scoring='accuracy').mean()
# SVC_CV_predictions = cross_val_predict(Base_SVC_Model_Pipeline, X, Y, cv=5)

# # Calculate recall, precision, and F1 score for each fold
# SVC_CV_recall = recall_score(Y, SVC_CV_predictions, average=None).mean()
# SVC_CV_precision = precision_score(Y, SVC_CV_predictions, average=None).mean()
# SVC_CV_f1_score = f1_score(Y, SVC_CV_predictions, average=None).mean()

# # Display the metrics
# print("Accuracy:", SVC_CV_accuracy)
# print("Recall:", SVC_CV_recall)
# print("Precision:", SVC_CV_precision)
# print("F1 Score:", SVC_CV_f1_score)

In [10]:
# #Create Pipeline to be used in Descision Tree Model
# Base_DT_Model_Pipeline = Pipeline(steps=[
#     ('Preprocessor', Preprocessor),
#     ('Feature_Selection', Selection),
#     ('DT_Model',DecisionTreeClassifier())])

# #Fit the training data 
# Base_DT_Model_Pipeline.fit(X_train, y_train)

# #Make Predictions on the test set 
# Base_DT_Model_Predictions = Base_DT_Model_Pipeline.predict(X_test)

# #Store individual metrics in a variable for later use 
# DT_Accuracy = (accuracy_score(y_test, Base_DT_Model_Predictions))*100
# DT_f_score = (f1_score(y_test, Base_DT_Model_Predictions))*100
# DT_recall_ = (recall_score(y_test, Base_DT_Model_Predictions))*100
# DT_precision = (precision_score(y_test, Base_DT_Model_Predictions))*100

# #Get Error Metrics of Random Forest Model
# DT_Model_Metrics = enhanced_metrics(y_test, Base_DT_Model_Predictions)

# #View Error Metric 
# print(DT_Model_Metrics)

In [11]:
# Create Model Pipeline
Base_RF_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('Feature_Selection', Selection),
    ('RF_Model', RandomForestClassifier(random_state=42))  
])

# Perform cross-validation
RF_CV_accuracy = cross_val_score(Base_RF_Model_Pipeline, X, Y, cv=10, scoring='accuracy').mean()
RF_CV_predictions = cross_val_predict(Base_RF_Model_Pipeline, X, Y, cv=10)

# Calculate recall, precision, and F1 score for each fold
RF_CV_recall = recall_score(Y, RF_CV_predictions, average=None).mean()
RF_CV_precision = precision_score(Y, RF_CV_predictions, average=None).mean()
RF_CV_f1_score = f1_score(Y, RF_CV_predictions, average=None).mean()

# Display the metrics
print("Accuracy:", RF_CV_accuracy)
print("Recall:", RF_CV_recall)
print("Precision:", RF_CV_precision)
print("F1 Score:", RF_CV_f1_score)


Accuracy: 0.5549319727891157
Recall: 0.43522615535889875
Precision: 0.42342342342342343
F1 Score: 0.4272358732522545


### Grid Search For Random Forest Model

In [12]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to use for the grid search
param_grid = {
    'RF_Model__n_estimators': [25,50, 100, 200],  # Number of trees in the forest
    'RF_Model__criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    'RF_Model__max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'RF_Model__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'RF_Model__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'RF_Model__max_features': ['sqrt', 'log2', None],  # Number of features to consider when looking for the best split
    # Note: For max_features, None means that max_features=n_features
    'RF_Model__bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
    'RF_Model__ccp_alpha': [0.0, 0.01, 0.1],  # Complexity parameter used for Minimal Cost-Complexity Pruning
    'RF_Model__max_samples': [None, 0.5, 0.75]  # If bootstrap is True, the number of samples to draw from X to train each base estimator
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=Base_RF_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Assuming X_train and y_train are already defined
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)


Fitting 5 folds for each of 15552 candidates, totalling 77760 fits


/home/clivence/base_jupyter/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
25920 fits failed out of a total of 77760.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25920 fits failed with the following error:
Traceback (most recent call last):
  File "/home/clivence/base_jupyter/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/clivence/base_jupyter/venv/lib/python3.9/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/clivence/base_jupyter/venv/lib/python3.9/site-packages/sklearn/pipeline.py", line 420, in fi

Best parameters found:  {'RF_Model__bootstrap': False, 'RF_Model__ccp_alpha': 0.0, 'RF_Model__criterion': 'entropy', 'RF_Model__max_depth': 20, 'RF_Model__max_features': 'sqrt', 'RF_Model__max_samples': None, 'RF_Model__min_samples_leaf': 2, 'RF_Model__min_samples_split': 2, 'RF_Model__n_estimators': 100}
Best score found:  0.7031784005468216


### Grid Search Metrics For Random Forest Model

In [21]:
# Create Model Pipeline
Base_RF_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('Feature_Selection', Selection),
    ('RF_Model', RandomForestClassifier(random_state=42,
                                       
                                       ccp_alpha=  0.0,
                                       criterion ='entropy',
                                       max_depth= 20,
                                       max_features= 'sqrt',
                                       min_samples_leaf= 2,
                                       min_samples_split= 2,
                                       n_estimators= 100,
                                       bootstrap= False,
                                       max_samples= None,
))  
])

# Perform cross-validation
RF_CV_accuracy = cross_val_score(Base_RF_Model_Pipeline, X, Y, cv=10, scoring='accuracy').mean()
RF_CV_predictions = cross_val_predict(Base_RF_Model_Pipeline, X, Y, cv=10)

# Calculate recall, precision, and F1 score for each fold
RF_CV_recall = recall_score(Y, RF_CV_predictions, average=None).mean()
RF_CV_precision = precision_score(Y, RF_CV_predictions, average=None).mean()
RF_CV_f1_score = f1_score(Y, RF_CV_predictions, average=None).mean()

# Display the metrics
print("Accuracy:", RF_CV_accuracy)
print("Recall:", RF_CV_recall)
print("Precision:", RF_CV_precision)
print("F1 Score:", RF_CV_f1_score)

Accuracy: 0.5569727891156462
Recall: 0.45467674532940017
Precision: 0.4513094190140845
F1 Score: 0.45243897270723854


In [13]:
??RandomForestClassifier

Init signature:
RandomForestClassifier(
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None,
)
Source:        
class RandomForestClassifier(ForestClassifier):
    """
    A random forest classifier.

    A random forest is a meta estimator that fits a number of decision tree
    classifiers on various sub-samples of the dataset and uses averaging to
    improve the predictive accuracy and control over-fitting.
    Trees in the forest use the best split strategy, i.e. equivalent to passing
    `splitter="best"` to the underlying :class:`~sklearn.tree.DecisionTreeRegressor`.
    The sub-sample size is contr

In [13]:
# # Define the parameter grid, prefixing each parameter with the name of the pipeline step followed by two underscores
# param_grid = {
#     'GBC_Model__loss': ['deviance', 'exponential'],
#     'GBC_Model__learning_rate': [0.01, 0.1, 0.5],
#     'GBC_Model__n_estimators': [50, 100, 200],
#     'GBC_Model__subsample': [0.5, 0.75, 1.0],
#     'GBC_Model__criterion': ['friedman_mse', 'mse', 'mae'],
#     'GBC_Model__min_samples_split': [2, 5, 10],
#     'GBC_Model__min_samples_leaf': [1, 2, 4],
#     'GBC_Model__max_depth': [3, 5, 10],
#     'GBC_Model__min_impurity_decrease': [0.0, 0.1, 0.2],
#     'GBC_Model__max_features': ['auto', 'sqrt', 'log2', None],
#     'GBC_Model__warm_start': [True, False]
# }

# # Create GridSearchCV
# grid_search = GridSearchCV(estimator=Base_GBC_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_params = grid_search.best_params_

# print("Best parameters found:")
# print(best_params)

In [14]:
??RandomForestClassifier

In [15]:
# from sklearn.model_selection import GridSearchCV

# # Define the parameter grid to use for the grid search
# param_grid = {
#     'RF_Model__n_estimators': [10, 20,30,50,75, 100,150, 200],  # Number of trees in the forest
#     'RF_Model__criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
#     'RF_Model__max_depth': [None,5, 10,15, 20],  # Maximum depth of the tree
#     'RF_Model__min_samples_split': [2, 5, 10, 15,],  # Minimum number of samples required to split an internal node
#     'RF_Model__min_samples_leaf': [1,3, 5],  # Minimum number of samples required to be at a leaf node
#     'RF_Model__max_features': ['sqrt', 'log2', None],  # Number of features to consider when looking for the best split
#     # Note: For max_features, None means that max_features=n_features
#     'RF_Model__bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
#     'RF_Model__ccp_alpha': [0.0, 0.01, 0.1],  # Complexity parameter used for Minimal Cost-Complexity Pruning
#     'RF_Model__max_samples': [None,0.25, 0.5, 0.75]  # If bootstrap is True, the number of samples to draw from X to train each base estimator
# }

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=Base_RF_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# # Assuming X_train and y_train are already defined
# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Print the best parameters and the best score
# print("Best parameters found: ", grid_search.best_params_)
# print("Best score found: ", grid_search.best_score_)


In [16]:
# #Create Pipeline to be used in Gradient Boosting Classfier Model
# Base_GBC_Model_Pipeline = Pipeline(steps=[
#     ('Preprocessor', Preprocessor),
#     ('GBC_Model',GradientBoostingClassifier())])

# # Perform cross-validation
# GBC_CV_accuracy = cross_val_score(Base_GBC_Model_Pipeline, X, Y, cv=5, scoring='accuracy').mean()
# GBC_CV_predictions = cross_val_predict(Base_GBC_Model_Pipeline, X, Y, cv=5)

# # Calculate recall, precision, and F1 score for each fold
# GBC_CV_recall = recall_score(Y, GBC_CV_predictions, average=None).mean()
# GBC_CV_precision = precision_score(Y, GBC_CV_predictions, average=None).mean()
# GBC_CV_f1_score = f1_score(Y, GBC_CV_predictions, average=None).mean()

# # Display the metrics
# print("Accuracy:", GBC_CV_accuracy)
# print("Recall:", GBC_CV_recall)
# print("Precision:", GBC_CV_precision)
# print("F1 Score:", GBC_CV_f1_score)


In [17]:
#Create Pipeline to be used in Gradient Boosting Classfier Model
Base_GBC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('GBC_Model',GradientBoostingClassifier(
    learning_rate=0.1,
    n_estimators=100,
    subsample=.75,
    criterion='friedman_mse',
    min_samples_split=10,
    min_samples_leaf=4,
    max_depth=5,
    min_impurity_decrease=0.1,
    random_state=42,
    max_features='sqrt',
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
   ))])

# Perform cross-validation
GBC_CV_accuracy = cross_val_score(Base_GBC_Model_Pipeline, X, Y, cv=5, scoring='accuracy').mean()
GBC_CV_predictions = cross_val_predict(Base_GBC_Model_Pipeline, X, Y, cv=5)

# Calculate recall, precision, and F1 score for each fold
GBC_CV_recall = recall_score(Y, GBC_CV_predictions, average=None).mean()
GBC_CV_precision = precision_score(Y, GBC_CV_predictions, average=None).mean()
GBC_CV_f1_score = f1_score(Y, GBC_CV_predictions, average=None).mean()

# Display the metrics
print("Accuracy:", GBC_CV_accuracy)
print("Recall:", GBC_CV_recall)
print("Precision:", GBC_CV_precision)
print("F1 Score:", GBC_CV_f1_score)

Accuracy: 0.5629939862542955
Recall: 0.4670907079646017
Precision: 0.4654887218045113
F1 Score: 0.466013109566001


In [18]:
from sklearn.model_selection import cross_validate, StratifiedKFold

# Define StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define scoring metrics
scoring = ['accuracy', 'recall', 'precision', 'f1']

# Perform cross-validation
cv_results = cross_validate(Base_GBC_Model_Pipeline, X, Y, cv=skf, scoring=scoring)

# Calculate the mean of each metric
accuracy_mean = cv_results['test_accuracy'].mean()
recall_mean = cv_results['test_recall'].mean()
precision_mean = cv_results['test_precision'].mean()
f1_mean = cv_results['test_f1'].mean()

print("StratifiedKFold CV Results:")
print(f"Accuracy: {accuracy_mean}")
print(f"Recall: {recall_mean}")
print(f"Precision: {precision_mean}")
print(f"F1 Score: {f1_mean}")


StratifiedKFold CV Results:
Accuracy: 0.6833118556701031
Recall: 0.8967954345917472
Precision: 0.7205529209160536
F1 Score: 0.7990249871289621


In [19]:
from sklearn.model_selection import ShuffleSplit

# Define ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Perform cross-validation
cv_results = cross_validate(Base_GBC_Model_Pipeline, X, Y, cv=ss, scoring=scoring)

# Calculate the mean of each metric
accuracy_mean = cv_results['test_accuracy'].mean()
recall_mean = cv_results['test_recall'].mean()
precision_mean = cv_results['test_precision'].mean()
f1_mean = cv_results['test_f1'].mean()

print("ShuffleSplit CV Results:")
print(f"Accuracy: {accuracy_mean}")
print(f"Recall: {recall_mean}")
print(f"Precision: {precision_mean}")
print(f"F1 Score: {f1_mean}")


In [20]:
import numpy as np

In [21]:
# import numpy as np

# # Given values
# values = [0.01, 0.1, 0.5]

# # Using list comprehension with numpy.array to recreate the list
# list_from_numpy = [np.array(values)[i] for i in range(len(values))]

# print(list_from_numpy)


In [22]:
# # Define the parameter grid, prefixing each parameter with the name of the pipeline step followed by two underscores
# param_grid = {
#     'GBC_Model__loss': ['deviance', 'exponential'],
#     'GBC_Model__learning_rate': [0.01, 0.1, 0.5],
#     'GBC_Model__n_estimators': [50, 100, 200],
#     'GBC_Model__subsample': [0.5, 0.75, 1.0],
#     'GBC_Model__criterion': ['friedman_mse', 'mse', 'mae'],
#     'GBC_Model__min_samples_split': [2, 5, 10],
#     'GBC_Model__min_samples_leaf': [1, 2, 4],
#     'GBC_Model__max_depth': [3, 5, 10],
#     'GBC_Model__min_impurity_decrease': [0.0, 0.1, 0.2],
#     'GBC_Model__max_features': ['auto', 'sqrt', 'log2', None],
#     'GBC_Model__warm_start': [True, False]
# }

# # Create GridSearchCV
# grid_search = GridSearchCV(estimator=Base_GBC_Model_Pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_params = grid_search.best_params_

# print("Best parameters found:")
# print(best_params)

In [23]:
# grid = GridSearchCV(Base_GBC_Model_Pipeline, params, cv=5, scoring='accuracy')
# grid.fit(X,Y)

In [24]:
# ??GradientBoostingClassifier

In [25]:
# Best parameters found:
# {'GBC_Model__criterion': 'friedman_mse',
#  'GBC_Model__learning_rate': 0.1, 
#  'GBC_Model__loss': 'exponential',
#  'GBC_Model__max_depth': 5,
#  'GBC_Model__max_features': 'sqrt', 
#  'GBC_Model__min_impurity_decrease': 0.1,
#  'GBC_Model__min_samples_leaf': 4, 
#  'GBC_Model__min_samples_split': 10,
#  'GBC_Model__n_estimators': 100,
#  'GBC_Model__subsample': 0.75,
#  'GBC_Model__warm_start': False}

In [26]:
#Create PiPeline to be used in Model
Base_ADAC_Model_Pipeline = Pipeline(steps=[
    ('Preprocessor', Preprocessor),
    ('RF_Model',AdaBoostClassifier())])

#Fit the training data 
Base_ADAC_Model_Pipeline.fit(X_train, y_train)

#Make Predictions on the test set 
Base_ADAC_Model_Predictions = Base_ADAC_Model_Pipeline.predict(X_test)

#Store individual metrics in a variable for later use 
ADAC_Accuracy = (accuracy_score(y_test, Base_ADAC_Model_Predictions))*100
ADAC_f_score = (f1_score(y_test, Base_ADAC_Model_Predictions))*100
ADAC_recall_ = (recall_score(y_test, Base_ADAC_Model_Predictions))*100
ADAC_precision = (precision_score(y_test, Base_ADAC_Model_Predictions))*100


#Get Error Metrics of Random Forest Model
ADAC_Model_Metrics = enhanced_metrics(y_test, Base_ADAC_Model_Predictions)

#View Error Metric 
print(ADAC_Model_Metrics)

### Base Model Summary 

In [ ]:
#Create a dataframe to score metric scores
Classifier_Scores = pd.DataFrame()

In [ ]:
# Create a DataFrame to store these values
metrics_df = pd.DataFrame({
    'Logistic Regression': [LR_Accuracy, LR_f_score, LR_recall_, LR_precision],
    'Descision Tree' : [DT_Accuracy, DT_f_score, DT_recall_, DT_precision],
    'Support Vector Machine' : [SVC_Accuracy, SVC_f_score, SVC_recall_, SVC_precision],
    'Random Forest' : [RF_Accuracy, RF_f_score, RF_recall_, RF_precision],
    'Gradient Boosting' : [GBC_Accuracy, GBC_f_score, GBC_recall_, GBC_precision],
    'ADA Boost ' : [ADAC_Accuracy, ADAC_f_score, ADAC_recall_, ADAC_precision],
}, index=['Accuracy','F1 Score', 'Recall', 'Precision'])

# Transpose the DataFrame 
metrics_df = metrics_df.transpose()

#rename the index 
metrics_df.index.name='Model Name'

metrics_df.reset_index(inplace=True)

# # Display the DataFrame
print(metrics_df)

In [ ]:
#Sort the df based on each metric 
accuracy_sorted_df = metrics_df.sort_values('Accuracy',ascending=False)
f1_sorted_df = metrics_df.sort_values('F1 Score',ascending=False)
recall_sorted_df = metrics_df.sort_values('Recall',ascending=False)
precision_sorted_df = metrics_df.sort_values('Precision',ascending=False)

#Create Figure 
fig, axs= plt.subplots(1,4, figsize=(24,10))

fig.suptitle("Base Model Performance",fontsize=16)

#Plot Accuracy in AXS[0,0]
bar_accuracy = axs[0].bar(accuracy_sorted_df['Model Name'], accuracy_sorted_df['Accuracy'])
axs[0].set_title('Accuracy By Classifiers')
axs[0].set_ylabel('Percentage %')
axs[0].set_xticklabels(accuracy_sorted_df['Model Name'], rotation=45)
for bar in bar_accuracy:
    height = bar.get_height()
    axs[0].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')

#Plot Precision in AXS[1[]
bar_precision = axs[1].bar(precision_sorted_df['Model Name'], precision_sorted_df['Precision'])
axs[1].set_title('Precision By Classifiers')
axs[1].set_ylabel('Percentage %')
axs[1].set_xticklabels(precision_sorted_df['Model Name'], rotation=45)
for bar in bar_precision:
    height = bar.get_height()
    axs[1].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')

#Plot Recall in AXS[2]
bar_recall = axs[2].bar(recall_sorted_df['Model Name'], recall_sorted_df['Recall'])
axs[2].set_title('Recall By Classifiers')
axs[2].set_ylabel('Percentage %')
axs[2].set_xticklabels(recall_sorted_df['Model Name'], rotation=45)
for bar in bar_recall:
    height = bar.get_height()
    axs[2].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')
    
#Plot F1 Score in AXS[3]
bar_fscore = axs[3].bar(f1_sorted_df['Model Name'], f1_sorted_df['F1 Score'])
axs[3].set_title('F1 Score By Classifiers')
axs[3].set_ylabel('Percentage %')
axs[3].set_xticklabels(f1_sorted_df['Model Name'], rotation=45)
for bar in bar_fscore:
    height = bar.get_height()
    axs[3].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}%', ha='center', va='bottom')
    
plt.tight_layout()


### There might be an issue between the Recall and Precision value of Logistic Regression - do not delete this until investigate further 

### Feature Importance

In [ ]:
#Retrived the Feature Selection Step 
feature_selection = Base_RF_Model_Pipeline.named_steps['Feature_Selection']

#Retrieve the trained model 
trained_RF_Model = Base_RF_Model_Pipeline.named_steps['RF_Model']

feature_importances = trained_RF_Model.feature_importances_

# Get the indices of selected features
selected_feature_indices = feature_selection.get_support(indices=True)

# Extract feature names
all_feature_names = Preprocessor.get_feature_names_out()
selected_feature_names = [all_feature_names[i] for i in selected_feature_indices]

# Combine feature names with their importances
features_and_importances = zip(selected_feature_names, feature_importances)

# Sort features by importance
sorted_features_and_importances = sorted(features_and_importances, key=lambda x: x[1], reverse=True)


# Create a DataFrame from the features and importances
feature_importance_df = pd.DataFrame(sorted_features_and_importances, columns=['Feature', 'Importance'])

# Display the DataFrame
feature_importance_df.head(20)

### Reference: Concepts To Understand 

#### Accuracy: 
                - Measures the overall correctness of the classifier. It's calculated by dividing the number of correctly classifier samples by the total number of samples. 
                - Accuracy tells us how often the classifier correctly predicted whether a trade is a successful or not compared to the total number of trades.
                - An Accuracy of .76 means that classifier correctly predicted the trade status for approximately 76% of the time. 
                
#### Recall:
             - Also known as sensitivity or true positive, measures  the ability of the classifier to correctly identify all positive instances. 
             - In the context of a trade status classifier, recall tells us how often the classifier correctly identified out of all successful trades.
             - A recall of approximately .795 means that the classifier correctly identified 79.5% of the successful trades. 
#### Precision 
              - Measures the ability of the classifier to correctly identify only relevant instances among all instances it has classified as positive. 
              - In the context of a trade status classifier, precision measures the proportion of correctly predicted successful trades out of all successful trades.
              - A precision of approximately .933 means that about 93.3% of the trades predicted as successful by the classifier were actually successful. 
#### F1 Score
                - Is a metric that combines precision and recall into a single value - it is the harmonic mean between precision and recall. 
                - Precision measures the proportion of correctly predicted positive cases among all predicted cases.
                - While Recall, measures the proportion of correctly predicted positive cases among all actual positive cases.
                - F1 Score provides a balance between precision and recall, it is especially useful when there is an uneven class distribution.
                - In the context of a trade status classifier, an F1 Score of .859 indicate a good balance between precision and recall predicting whether trades were successful or not. 